# Model 3: The Production Mix Problem II

$$
\begin{array}{rl}
    \max  & 5\, x_s + 20\, x_l \\[5pt]
    \text{s.t.} & x_s + 3\, x_l  \leq 200 \\
    & 3\, x_s + 2\, x_l \leq 160 \\[5pt]
    & x_1,x_2,x_3 \in \{0, 1, 2, \ldots\} 
\end{array}
$$

## The concrete Pyomo model

### Import required libraries and create the model object

In [ ]:
!pip install gurobipy pyomo

solver_options = {
    "WLSACCESSID": "...",  # your WSL access id (string)
    "WLSSECRET": "...",  # your WSL secret (string)
    "LICENSEID": ...,  # your license id (integer)
}

In [ ]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

mod = pyo.ConcreteModel(name="production_mix_2")

### Define the components of the model

In [ ]:
# decision variables:
mod.x_s = pyo.Var(name="num_of_small_boards", domain=pyo.NonNegativeIntegers)
mod.x_l = pyo.Var(name="num_of_large_boards", domain=pyo.NonNegativeIntegers)

# objective function:
mod.obj = pyo.Objective(expr=5 * mod.x_s + 20 * mod.x_l, sense=pyo.maximize)

# constraints:
mod.con_wood = pyo.Constraint(expr=mod.x_s + 3 * mod.x_l <= 200, name="wood_resource")
mod.con_time = pyo.Constraint(expr=3 * mod.x_s + 2 * mod.x_l <= 160, name="time_resource")

### Solve the model

One of the options you can use is `tee=True` when calling the `solve()` function. This can help you see the progress of the solver, such as the value of the objective function, the number of iterations, the MIP gap, solution time, etc.

In [ ]:
opt = SolverFactory("gurobi", solver_io="python", manage_env=True, options=solver_options)
result = opt.solve(mod, tee=True)

### Display and interpret the results

To access the different components of a Pyomo model, such as variables and constraints, we can use the `component_objects()` method. This method returns a generator object that allows us to loop over the specified component type. In the following code, we want to display the optimal value of the decision variables in the model as well as the consumed resources.

In [ ]:
print("Objective value =", pyo.value(mod.obj))

# retrieve the active variables in the model
variables = mod.component_objects(pyo.Var, active=True)

# iterate over variables
for var in variables:
    # print their name and final value
    print(var.name, "=", pyo.value(var))

# retrieve the active constraints
constraints = mod.component_objects(pyo.Constraint, active=True)

# iterate over constraints
for con in constraints:
    # print their left-hand-side value
    print("consumed", con.name, "=", pyo.value(con))
    # pyo.value(con) is equivalent to con() and con.body()